In [ ]:
# ============================================================================
# 전력 수요 예측 프로젝트 - 데이터 전처리 (개선 버전)
# ============================================================================

import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta

# =============================================================================
# 1단계: 법정동 데이터 전처리
# =============================================================================
print("=" * 60)
print("1단계: 법정동 데이터 전처리")
print("=" * 60)

file_path = "data/법정동코드 조회자료.xls"
dong_df = pd.read_excel(file_path)
print(f"✓ 법정동 데이터 로드 완료: {dong_df.shape}")

# 전체코드 분리 (앞 5자리/뒤 5자리)
dong_df["법정동코드"] = dong_df["법정동코드"].astype(str).str.zfill(10)
dong_df["시군구코드"] = dong_df["법정동코드"].str[:5]
dong_df["법정동코드_하위"] = dong_df["법정동코드"].str[5:]

# 상위 행정구역 제거
before = len(dong_df)
dong_df = dong_df[dong_df["법정동코드_하위"] != "00000"].copy()
print(f"✓ 상위 행정구역 제거: {before} → {len(dong_df)}")

# 법정동 전체명 분리
dong_df["법정동명"] = dong_df["법정동명"].astype(str).str.strip()
dong_df["시군구명"] = dong_df["법정동명"].apply(lambda x: " ".join(x.split()[:-1]))
dong_df["법정동명_하위"] = dong_df["법정동명"].apply(lambda x: x.split()[-1])

dong_clean = dong_df[[
    "법정동코드", "시군구코드", "법정동코드_하위", 
    "시군구명", "법정동명_하위"
]].drop_duplicates().reset_index(drop=True)

dong_clean.to_csv("data/법정동_마스터.csv", index=False, encoding="utf-8-sig")
print(f"✓ 법정동 마스터 생성: {dong_clean.shape}\n")


# =============================================================================
# 2단계: 전력량 데이터 전처리 (성능 최적화)
# =============================================================================
print("=" * 60)
print("2단계: 전력량 데이터 전처리 (개선 버전)")
print("=" * 60)

# 2-1. 전력량 데이터 로드
power_path = "data/법정동별시간별전력사용량.csv"
try:
    power_df = pd.read_csv(power_path, encoding="utf-8-sig")
except UnicodeDecodeError:
    power_df = pd.read_csv(power_path, encoding="cp949")
print(f"✓ 전력량 데이터 로드: {power_df.shape}")

# 2-2. 중복 제거
before = len(power_df)
power_df = power_df.drop_duplicates().copy()
print(f"✓ 중복 제거: {before} → {len(power_df)}")

# 2-3. 법정동코드 생성 및 일시 변환
power_df["SIGUNGU_CD"] = power_df["SIGUNGU_CD"].astype(str).str.zfill(5)
power_df["BJDONG_CD"] = power_df["BJDONG_CD"].astype(str).str.zfill(5)
power_df["법정동코드"] = power_df["SIGUNGU_CD"] + power_df["BJDONG_CD"]

power_df["USE_YM"] = power_df["USE_YM"].astype(str)
power_df["USE_HM"] = power_df["USE_HM"].astype(str).str.zfill(4)
power_df["일시"] = pd.to_datetime(
    power_df["USE_YM"].str[:8] + power_df["USE_HM"],
    format="%Y%m%d%H%M", 
    errors="coerce"
)

# 전력량 숫자 변환
power_df["FDRCT_VLD_KWH"] = pd.to_numeric(power_df["FDRCT_VLD_KWH"], errors="coerce")

# 2-4. 시간 정합성 검증 (중복 처리)
print("\n[시간 정합성 검증]")
time_check = power_df.groupby(["법정동코드", "일시"]).size()
duplicates = time_check[time_check > 1]
if len(duplicates) > 0:
    print(f"⚠️ 중복 시간 발견: {len(duplicates)}건 → 평균값으로 집계")
    power_df = power_df.groupby(["법정동코드", "일시"], as_index=False)["FDRCT_VLD_KWH"].mean()
else:
    print("✓ 중복 시간 없음")

# 2-5. 시간별 데이터 구조화
power_hourly = power_df.copy()
power_hourly["일자"] = power_hourly["일시"].dt.date
power_hourly["시간"] = power_hourly["일시"].dt.hour
power_hourly["년"] = power_hourly["일시"].dt.year
power_hourly["월"] = power_hourly["일시"].dt.month

print(f"✓ 시간별 데이터 생성: {power_hourly.shape}")

# 2-6. 🔥 이상치 완화 - Winsorizing (벡터화 버전)
print("\n[전력량 이상치 완화 - Winsorizing]")
print("📝 Note: Winsorizing 기준(quantile)은 Train 데이터 기준으로만 산출해야 함")

# 법정동별 0.5%, 99.5% 분위수 계산
q_values = power_hourly.groupby("법정동코드")["FDRCT_VLD_KWH"].quantile([0.005, 0.995]).unstack()
q_values.columns = ["lower_bound", "upper_bound"]

# 병합 후 clip
power_hourly = power_hourly.merge(q_values, on="법정동코드", how="left")

power_hourly["전력량_원본"] = power_hourly["FDRCT_VLD_KWH"]
power_hourly["전력량_완화"] = power_hourly["FDRCT_VLD_KWH"].clip(
    lower=power_hourly["lower_bound"],
    upper=power_hourly["upper_bound"]
)

# 임시 컬럼 삭제
power_hourly.drop(columns=["lower_bound", "upper_bound"], inplace=True)

print("✓ Winsorizing 완료 (법정동별 0.5~99.5% 기준)")
print(f"  - 원본 vs 완화 차이: {(power_hourly['전력량_원본'] != power_hourly['전력량_완화']).sum():,}건")

# 2-7. 🔥 저전력 플래그 생성 (벡터화 버전)
print("\n[저전력 플래그 생성]")

# 법정동별 하위 2% 임계값 계산
low_threshold = power_hourly.groupby("법정동코드")["전력량_완화"].quantile(0.02)
low_threshold.name = "저전력_임계값"

# 병합 후 플래그 생성
power_hourly = power_hourly.merge(low_threshold, on="법정동코드", how="left")
power_hourly["저전력_플래그"] = (
    power_hourly["전력량_완화"] < power_hourly["저전력_임계값"]
).astype(int)

# 임시 컬럼 삭제
power_hourly.drop(columns=["저전력_임계값"], inplace=True)

print(f"✓ 저전력 플래그 생성 완료 (하위 2% 기준)")
print(f"  - 저전력 데이터: {power_hourly['저전력_플래그'].sum():,}건 ({power_hourly['저전력_플래그'].mean()*100:.2f}%)")

# 2-8. 법정동 병합
merged_power = power_hourly.merge(dong_clean, on="법정동코드", how="left")
print(f"✓ 법정동 마스터와 병합: {merged_power.shape}")

merged_power.to_csv("data/전력_시간별_병합.csv", index=False, encoding="utf-8-sig")
print("✓ 저장: data/전력_시간별_병합.csv\n")


# =============================================================================
# 3단계: 날씨 데이터 전처리
# =============================================================================
print("=" * 60)
print("3단계: 날씨 데이터 전처리")
print("=" * 60)

base_dir = "data/weather_data"

# 3-1. 시간별(HR) 날씨 데이터 로드
hr_files = [
    "SURFACE_ASOS_108_HR_2022_2022_2023.csv",
    "SURFACE_ASOS_108_HR_2023_2023_2024.csv",
    "SURFACE_ASOS_108_HR_2024_2024_2025.csv"
]

hr_dfs = []
for f in hr_files:
    fpath = os.path.join(base_dir, f)
    if not os.path.exists(fpath):
        print(f"⚠️ 파일 없음: {f}")
        continue
    try:
        df = pd.read_csv(fpath, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(fpath, encoding="cp949")
    hr_dfs.append(df)

if not hr_dfs:
    raise FileNotFoundError("시간별 날씨 데이터를 찾을 수 없습니다.")

weather_hr = pd.concat(hr_dfs, ignore_index=True)
print(f"✓ 시간별 날씨 데이터 로드: {weather_hr.shape}")

# 3-2. 날짜/시간 컬럼 탐지
date_col = None
for c in weather_hr.columns:
    if any(k in str(c).lower() for k in ["tm", "일시", "time"]):
        s = pd.to_datetime(weather_hr[c], errors="coerce")
        if s.notna().sum() > len(weather_hr) * 0.9:
            date_col = c
            weather_hr["일시"] = s
            break

if date_col is None:
    raise ValueError("날씨 데이터에서 날짜 컬럼을 찾을 수 없습니다.")

weather_hr["일자"] = weather_hr["일시"].dt.date
weather_hr["시간"] = weather_hr["일시"].dt.hour
print(f"✓ 날짜/시간 변환 완료: {date_col} → 일시")

# 3-3. 필요한 컬럼 매핑
col_mapping = {
    "기온(°C)": "기온",
    "강수량(mm)": "강수량", 
    "풍속(m/s)": "풍속",
    "습도(%)": "습도"
}

use_cols = ["일시", "일자", "시간"]
for orig, new in col_mapping.items():
    if orig in weather_hr.columns:
        weather_hr[new] = pd.to_numeric(weather_hr[orig], errors="coerce")
        use_cols.append(new)

weather_clean = weather_hr[use_cols].copy()
print(f"✓ 날씨 컬럼 선택: {[c for c in use_cols if c not in ['일시', '일자', '시간']]}")

# 3-4. 결측치 처리
print("\n[결측치 처리]")
if "강수량" in weather_clean.columns:
    weather_clean["강수량"] = weather_clean["강수량"].fillna(0)
    print("✓ 강수량 결측치 → 0")

for col in ["기온", "풍속", "습도"]:
    if col in weather_clean.columns:
        before_na = weather_clean[col].isna().sum()
        weather_clean[col] = weather_clean[col].interpolate(method="linear")
        weather_clean[col] = weather_clean[col].fillna(method="ffill").fillna(method="bfill")
        print(f"✓ {col} 결측치 보간: {before_na} → {weather_clean[col].isna().sum()}")

# 3-5. 체감 관련 파생 변수
print("\n[기상 파생 변수 생성]")

if "기온" in weather_clean.columns and "습도" in weather_clean.columns:
    T = weather_clean["기온"]
    RH = weather_clean["습도"]
    
    # 불쾌지수
    weather_clean["불쾌지수"] = 0.81 * T + 0.01 * RH * (0.99 * T - 14.3) + 46.3
    print("✓ 불쾌지수 생성")
    
    # 냉난방도일 (Cooling/Heating Degree Days)
    weather_clean["냉방도일"] = (weather_clean["기온"] - 24).clip(lower=0)
    weather_clean["난방도일"] = (18 - weather_clean["기온"]).clip(lower=0)
    print("✓ 냉난방도일 생성 (냉방기준: 24°C, 난방기준: 18°C)")

# 3-6. 기온 변화량
if "기온" in weather_clean.columns:
    weather_clean = weather_clean.sort_values("일시").reset_index(drop=True)
    weather_clean["기온변화_1h"] = weather_clean["기온"].diff(1).fillna(0)
    weather_clean["기온변화_24h"] = weather_clean["기온"].diff(24).fillna(0)
    print("✓ 기온 변화량 생성 (1h, 24h)")

# 3-7. 강수 여부 플래그
if "강수량" in weather_clean.columns:
    weather_clean["강수여부"] = (weather_clean["강수량"] > 0).astype(int)
    print("✓ 강수 여부 플래그 생성")

# 시군구코드 부여
weather_clean["시군구코드"] = "11000"

weather_clean.to_csv("data/날씨_시간별_정제.csv", index=False, encoding="utf-8-sig")
print("✓ 저장: data/날씨_시간별_정제.csv\n")


# =============================================================================
# 4단계: 최종 통합
# =============================================================================
print("=" * 60)
print("4단계: 최종 데이터 통합")
print("=" * 60)

# 4-1. 전력 + 날씨 병합
final_df = merged_power.merge(
    weather_clean,
    on=["일자", "시간", "시군구코드"],
    how="left",
    suffixes=("", "_날씨")
)
print(f"✓ 전력+날씨 병합 완료: {final_df.shape}")

# 4-2. 시간 파생 변수 (이미 전력 데이터에 존재)
print("\n[시간 파생 변수 생성]")

# 일시 확인 및 복구
if "일시" not in final_df.columns or final_df["일시"].isna().all():
    final_df["일시"] = pd.to_datetime(final_df["일자"]) + pd.to_timedelta(final_df["시간"], unit='h')

# 년월일 (이미 있으면 스킵)
if "년" not in final_df.columns:
    final_df["년"] = final_df["일시"].dt.year
if "월" not in final_df.columns:
    final_df["월"] = final_df["일시"].dt.month
if "일" not in final_df.columns:
    final_df["일"] = final_df["일시"].dt.day

final_df["요일"] = final_df["일시"].dt.dayofweek  # 0=월요일
final_df["주말여부"] = (final_df["요일"] >= 5).astype(int)

# 4-3. 공휴일 플래그
holidays_2022_2025 = [
    # 2022년
    "2022-01-01", "2022-01-31", "2022-02-01", "2022-02-02", "2022-03-01",
    "2022-03-09", "2022-05-05", "2022-05-08", "2022-06-06", "2022-08-15",
    "2022-09-09", "2022-09-10", "2022-09-11", "2022-10-03", "2022-10-09",
    "2022-10-10", "2022-12-25",
    # 2023년
    "2023-01-01", "2023-01-21", "2023-01-22", "2023-01-23", "2023-01-24",
    "2023-03-01", "2023-05-05", "2023-05-27", "2023-06-06", "2023-08-15",
    "2023-09-28", "2023-09-29", "2023-09-30", "2023-10-03", "2023-10-09",
    "2023-12-25",
    # 2024년
    "2024-01-01", "2024-02-09", "2024-02-10", "2024-02-11", "2024-02-12",
    "2024-03-01", "2024-04-10", "2024-05-05", "2024-05-06", "2024-05-15",
    "2024-06-06", "2024-08-15", "2024-09-16", "2024-09-17", "2024-09-18",
    "2024-10-03", "2024-10-09", "2024-12-25",
    # 2025년
    "2025-01-01", "2025-01-28", "2025-01-29", "2025-01-30", "2025-03-01",
    "2025-03-03", "2025-05-05", "2025-05-06", "2025-06-06", "2025-08-15",
    "2025-10-03", "2025-10-05", "2025-10-06", "2025-10-07", "2025-10-08",
    "2025-10-09", "2025-12-25"
]

holidays_set = set(pd.to_datetime(holidays_2022_2025).date)
final_df["일자_temp"] = pd.to_datetime(final_df["일자"]).dt.date
final_df["공휴일여부"] = final_df["일자_temp"].isin(holidays_set).astype(int)
final_df.drop("일자_temp", axis=1, inplace=True)

print("✓ 시간 파생 변수 완료 (년월일, 요일, 주말, 공휴일)")

# 4-4. 법정동별 통계 Feature (Train 기간 기준)
print("\n[법정동별 통계 Feature 생성]")
print("📝 Note: Train 기간(2022~2024) 기준으로만 계산 (Data Leakage 방지)")

train_mask = final_df["년"] <= 2024
train_data = final_df[train_mask].copy()

# 법정동별 기본 통계
dong_stats = train_data.groupby("법정동코드")["전력량_완화"].agg(["mean", "std", "max"]).reset_index()
dong_stats.columns = ["법정동코드", "법정동_평균전력", "법정동_표준편차", "법정동_최대전력"]

# 법정동×시간대별 평균
dong_hour_stats = train_data.groupby(["법정동코드", "시간"])["전력량_완화"].mean().reset_index()
dong_hour_stats.columns = ["법정동코드", "시간", "법정동_시간평균전력"]

# 병합
final_df = final_df.merge(dong_stats, on="법정동코드", how="left")
final_df = final_df.merge(dong_hour_stats, on=["법정동코드", "시간"], how="left")

print("✓ 법정동 통계 Feature 병합 완료")
print(f"  - 법정동_평균전력, 표준편차, 최대전력")
print(f"  - 법정동_시간평균전력 (시간대별)")

# 4-5. 최종 저장
final_df.to_csv("data/최종_통합데이터_시간별.csv", index=False, encoding="utf-8-sig")

print("\n" + "=" * 60)
print("✅ 전처리 완료!")
print("=" * 60)
print(f"📊 최종 데이터 정보:")
print(f"  - 저장 경로: data/최종_통합데이터_시간별.csv")
print(f"  - 총 데이터: {len(final_df):,}개")
print(f"  - 컬럼 수: {len(final_df.columns)}개")
print(f"  - 기간: {pd.to_datetime(final_df['일자']).min().date()} ~ {pd.to_datetime(final_df['일자']).max().date()}")
print(f"  - 법정동 수: {final_df['법정동코드'].nunique()}개")

# 데이터 품질 체크
print("\n📋 데이터 품질 체크:")
null_summary = final_df.isnull().sum()
if null_summary.sum() > 0:
    print("  결측치 현황 (상위 5개):")
    for col, count in null_summary[null_summary > 0].head().items():
        print(f"    - {col}: {count:,}개 ({count/len(final_df)*100:.2f}%)")
else:
    print("  ✓ 결측치 없음")

print("\n📈 주요 변수 통계:")
key_cols = ["전력량_완화", "기온", "불쾌지수"]
for col in key_cols:
    if col in final_df.columns:
        print(f"\n  {col}:")
        print(f"    평균: {final_df[col].mean():.2f}")
        print(f"    표준편차: {final_df[col].std():.2f}")
        print(f"    최소: {final_df[col].min():.2f}")
        print(f"    최대: {final_df[col].max():.2f}")

print("\n✅ 전체 전처리 프로세스 완료!")